In [ ]:
##########################################################################################
################################################################ Указать название скрипта
script = "__template-agency.ipynb"

In [ ]:
##################################################################################
################################################################ импорт библиотек
import sys
import os
import configparser
import pandas as pd

# windows 10
fpath = os.path.join(os.path.dirname(script), 'patterns') 
sys.path.append(fpath)
config = configparser.ConfigParser()
config.read("__fields.ini")
config_db = configparser.ConfigParser()
config_db.read("__settings.ini")

# linux server
# sys.path.append(r'/var/home/rb1/!py_script/patterns')
# sys.path.append(r'/var/home/rb1/!py_script')
# config = configparser.ConfigParser()
# config.read(r'/var/home/rb1/!py_script/__data_fields.ini')
# config_db = configparser.ConfigParser()
# config_db.read(r'/var/home/rb1/!py_script/__settings.ini')

import yd
import ym
import patterns.click as click
import yesterday

from sqlalchemy import create_engine
from google.oauth2 import service_account

In [ ]:
###########################################################################################
################################################################ укажите параметры скрипта
 # начальная дата
dateTo = str(yesterday.getYesterday()) #конечная дата
dataset = "agency_riverstart" # целевой датасет или база данных для записи

mode_write = "replace" # перезапись данных
#mode_write = "append" # добавление данных

# GoogleBigQuery - коннект
# project_id = 'powerbi4rb'
# json_path = r"patterns\\pbi4rb_gbq.json" # windows 10
# json_path = r'/var/home/rb1/!py_script/patterns/pbi4rb_gbq.json' # linux

# PostgreSQL - коннект
engine = create_engine("postgresql://" + config_db['postgresql']['username'] + ':' + config_db['postgresql']['password'] + '@' + config_db['postgresql']['host'] + ':' + config_db['postgresql']['port'] + "/" + dataset)

In [ ]:
########################################################################################
################################################################ Итоговые датафреймы (в них собираются данные)
agency_costs = pd.DataFrame()
agency_conv = pd.DataFrame()

In [ ]:
# Нейминг клиента
name_client = "Пинчер СРМ"
dateFrom = "2023-04-05"

# Метрика - источники/конверсии. Обязательный пункт, может быть несколько счетчиков.
counter = [
    {'counter':'68370820', 
     'token':'XXXXXXXXXXXXXXXX', 
     'conv':[''],
     'dateFrom':dateFrom
     }
]

# Директ - расходы. Можно удалить если источник не используется
yd_clientLogin = [
    {'login':'pinchercrm-riverstart', 
        'token':'XXXXXXXXXXXXXXXX',
        'dateFrom':dateFrom
    },
    {'login':'pinscher-sales-client', 
        'token':'XXXXXXXXXXXXXXXX',
        'dateFrom':dateFrom
    }
]

# VK - click.ru расходы. Можно удалить если источник не используется
vk_c = [
    {'uid':'1675402',
     'tkn':'XXXXXXXXXXXXXXXXX',
     'dateFrom':dateFrom
     }
]

# MT - click.ru расходы. Можно удалить если источник не используется
mt_c = [
    {'uid':'1641088',
     'tkn':'XXXXXXXXXXXXXXXX',
     'dateFrom':dateFrom
     }
]

ga_c = [
    {'uid':'1654462',
     'tkn':'XXXXXXXXXXXXXXXX',
     'dateFrom':dateFrom
     }
]

In [ ]:
for i in counter:
    fields_dim = 'dim_agency' # набор параметров, которые требуется получить. для создания нового набора добавьте в файл data_fields.ini новый список полей для соответствующего источника
    fields_met = 'met_agency' # набор метрик, которые требуется получить. для создания нового набора добавьте в файл data_fields.ini новый список полей для соответствующего источника

    report_fields_dim = config['yandex_metrika_names'][fields_dim]
    report_fields_met = config['yandex_metrika_names'][fields_met]

    # goals_ym = ym.convert_goals_s(i['conv'])
    # report_fields_met = report_fields_met + ',' + goals_ym

    metrika_dataset = ym.get_ym_stat(i['counter'],"",i['token'],i['dateFrom'],dateTo,report_fields_met,report_fields_dim)
    value_cols = metrika_dataset.filter(like='ym_s_goal').columns
    metrika_dataset['Macroconversions'] = metrika_dataset[value_cols].sum(axis=1)
    metrika_dataset = metrika_dataset.drop(columns=value_cols)
    metrika_dataset['Client'] = name_client
    agency_conv = pd.concat([agency_conv, metrika_dataset], ignore_index=True)


In [ ]:
##############################################################################################################
#################################################### Сбор данных, закомментить ненужные источники

##############################################################################
############################################################### Яндекс директ

for i in yd_clientLogin:
    fields = 'yd_agency' # набор полей которые требуется получить. для создания нового набора добавьте в файл data_fields.ini новый список полей для соответствующего источника
    report_fields = list(config['yandex_direct_names'][fields].split(","))
    direct_dataset = yd.get_data_yd(i['token'], i['login'], "", i['dateFrom'], dateTo, 60, report_fields) # функция обращается к api директа и собирает данные
    direct_dataset['Client'] = name_client
    direct_dataset['TrafficType'] = 'Контекст'
    direct_dataset['service'] = 'YANDEX_DIRECT'
    agency_costs = pd.concat([agency_costs, direct_dataset], ignore_index=True) # кладем полученный датасет в общий, чтобы сохранить

In [ ]:
##############################################################################
############################################################### VK Ads CLICK.RU
for i in vk_c:
    vk_click_dataset = click.click_vk(i['dateFrom'],dateTo,i['uid'],i['tkn'])
    vk_click_dataset['Client'] = name_client
    vk_click_dataset = vk_click_dataset.rename(columns={'account':'Login'})
    vk_click_dataset = vk_click_dataset[['Date','Impressions','Clicks','Cost','Login','Client','TrafficType','service']]
    agency_costs = pd.concat([agency_costs, vk_click_dataset], ignore_index=True)

In [ ]:
##############################################################################
############################################################### MyTarget CLICK.RU
for i in mt_c:
    mt_click_dataset = click.click_mt(i['dateFrom'],dateTo,i['uid'],i['tkn'])
    mt_click_dataset['Client'] = name_client
    mt_click_dataset = mt_click_dataset.rename(columns={'account':'Login'})
    mt_click_dataset = mt_click_dataset[['Date','Impressions','Clicks','Cost','Login','Client','TrafficType','service']]
    agency_costs = pd.concat([agency_costs, mt_click_dataset], ignore_index=True)

In [ ]:
##############################################################################
############################################################### Google Ads CLICK.RU
for i in ga_c:
    ga_click_dataset = click.click_ga(i['dateFrom'],dateTo,i['uid'],i['tkn'])
    ga_click_dataset['Client'] = name_client
    ga_click_dataset = ga_click_dataset.rename(columns={'account':'Login'})
    ga_click_dataset = ga_click_dataset[['Date','Impressions','Clicks','Cost','Login','Client','TrafficType','service']]
    ga_click_dataset = click.usd_to_rub(i['dateFrom'],dateTo,ga_click_dataset)
    agency_costs = pd.concat([agency_costs, ga_click_dataset], ignore_index=True)

In [ ]:
###################################################################################################
################################################################ Отправляем полученные данные - GBQ
control_dataset.to_gbq(
          dataset + "." + "yd_optimize_camp" + "_stat",  # {dataset_name}.{table_name}
          project_id=project_id,
          if_exists=mode_write,  # fail // append // replace
          credentials=service_account.Credentials.from_service_account_file(json_path)
         )


In [ ]:
###################################################################################################
################################################################ Отправляем полученные данные - POSTGRESQL
# config = configparser.ConfigParser()
# config.read("settings.ini")

# # PostgreSQL - коннект
# engine = create_engine("postgresql://" + config_db['postgresql']['username'] + ':' + config_db['postgresql']['password'] + '@' + config_db['postgresql']['host'] + ':' + config_db['postgresql']['port'] + "/" + dataset)

# dataframe - название датафрейма с данными для записи
agency_costs.to_sql("agency_costs", engine, if_exists=mode_write)
agency_conv.to_sql("agency_conv", engine, if_exists=mode_write)

In [ ]:
df = pd.DataFrame(index=index, columns=['Client','Comment'])
df.to_sql("input_user", engine, if_exists=mode_write)